In [1]:
from tensorflow import keras
import tensorflow as tf
import argparse
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate

In [2]:
ds_train = tfds.load('horses_or_humans',split='train', as_supervised=True,shuffle_files=True)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteLPO9LD/horses_or_humans-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteLPO9LD/horses_or_humans-test.tfrecord


Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [3]:
ds_test = tfds.load('horses_or_humans',split='test', as_supervised=True,shuffle_files=True)

In [4]:
batch_size=50
input_shape = (224,224,3)

resizer = lambda x, y: (tf.image.resize(x, input_shape[:2]), y)

ds_train_batches = ds_train.map(resizer).cache().batch(batch_size).prefetch(buffer_size=10)
ds_test_batches = ds_test.map(resizer).cache().batch(batch_size).prefetch(buffer_size=10)

#records.map(lambda line: list(map(lambda x: x.replace("\"",""), line)))

In [5]:
target_size = (224,224) #(128,128)

In [8]:
def res_block_v2(filters_1 = 64, filters_2 = 128, filters_3 = 256,filters_4 = 512,filters_5 = 512,filters_6 = 4096,filters_7 = 4096,filters_8 = 1000):

  s=Input((*target_size,3))
  #x_1 = keras.layers.BatchNormalization()(input)
  c1_1 = keras.layers.Conv2D(filters=filters_1, kernel_size = (3,3),activation='relu', padding = 'same')(s)
  c1_2 = keras.layers.Conv2D(filters=filters_1, kernel_size = (3,3), activation='relu', padding = 'same')(c1_1)
  #x_1 = keras.layers.ReLU()(c1_2)
  p1 = MaxPooling2D(pool_size=(2,2),strides=(2,2)) (c1_2)
 
#################################pooling
  # x_3 = keras.layers.x_2(shape = (112,112,128))
  # x_3 = keras.layers.BatchNormalization()(x_2)
  # x_3 = keras.layers.ReLU()(x_3)
  c2_1 = keras.layers.Conv2D(filters=filters_2, kernel_size = (3,3),activation='relu', padding = 'same')(p1)
  c2_2 = keras.layers.Conv2D(filters=filters_2, kernel_size = (3,3),activation='relu', padding = 'same')(c2_1)
  p2 = MaxPooling2D(pool_size=(2,2),strides=(2,2)) (c2_2)


  c3_1 = keras.layers.Conv2D(filters=filters_3, kernel_size = (3,3),activation='relu', padding = 'same')(p2)
  c3_2 = keras.layers.Conv2D(filters=filters_3, kernel_size = (3,3),activation='relu', padding = 'same')(c3_1)
  c3_3 = keras.layers.Conv2D(filters=filters_3, kernel_size = (3,3),activation='relu', padding = 'same')(c3_2)
  p3 = MaxPooling2D(pool_size=(2,2),strides=(2,2)) (c3_3)

  #x_4 = keras.layers.ReLU()(x_4)
  c4_1 = keras.layers.Conv2D(filters=filters_4, kernel_size = (3,3),activation='relu', padding = 'same')(p3)
  c4_2 = keras.layers.Conv2D(filters=filters_4, kernel_size = (3,3),activation='relu', padding = 'same')(c4_1)
  c4_3 = keras.layers.Conv2D(filters=filters_4, kernel_size = (3,3),activation='relu', padding = 'same')(c4_2)
  p4 = MaxPooling2D(pool_size=(2,2),strides=(2,2)) (c4_3)

  #x_2 = keras.layers.BatchNormalization()(x_1)
  c5_1 = keras.layers.Conv2D(filters=filters_5, kernel_size = (3,3),activation='relu', padding = 'same')(p4)
  c5_2 = keras.layers.Conv2D(filters=filters_5, kernel_size = (3,3),activation='relu', padding = 'same')(c5_1)
  c5_3 = keras.layers.Conv2D(filters=filters_5, kernel_size = (3,3),activation='relu', padding = 'same')(c5_2)
  p5 = MaxPooling2D(pool_size=(2,2),strides=(2,2)) (c5_3)
  f5=keras.layers.Flatten()(p5)

  # c6_1=keras.layers.Conv2D(filters=filters_6, kernel_size = (1,1), padding = 'same')(p5)
  # c7_1=keras.layers.Conv2D(filters=filters_7, kernel_size = (1,1), padding = 'same')(c6_1)
  # c_out=keras.layers.Conv2D(filters=filters_8, kernel_size = (1,1),activation = 'softsign', padding = 'same')(c7_1)

  z=keras.layers.Dense(units=4096, activation='tanh')(f5)
  z=keras.layers.Dense(units=4096, activation='tanh')(z)
  out=keras.layers.Dense(units=1, activation = 'softsign')(z)



  return keras.models.Model(inputs=s,outputs=out)

In [9]:
model=res_block_v2()
model.compile(optimizer='adam',loss='bce')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 128)       0     

In [13]:
epochs = 10
patience = 5
model.fit(ds_train_batches, epochs=epochs, validation_data=ds_test_batches, callbacks = [keras.callbacks.EarlyStopping(patience=patience)])

Epoch 1/10
 6/21 [=======>......................] - ETA: 17:10 - loss: 1.6395

KeyboardInterrupt: ignored